# 120. User-Based Collaborative Filtering (협업 필터링)

### User-Based CF 의 알고리즘

| USER | M1 | M2 | M3 | M4 | M5 | U1과의 유사도 |
| --- | --- |--- |--- | ---|--- | --- |
| U1 | 2 | 5 | 3 |   |   |
|U2|4|4|3|5|1|0.19|
|U3|1|5|4| | 5| 0.89|
|U4|3|5|3|2|5  |0.94|
|U5|4|5|3|4| |0.65|
|U3, U4 평균|2|5|3.5|2|5|

1) U1과 취향이 비슷한 사용자를 찾는다. 취향은 각 사용자의 영화에 대한 평가의 유사성을 계산하여 찾는다. --> U3와 U4가 U1과 가장 높은 상관 관계를 보임.  

2) U1 과 가장 유사한 U3와 U4가 가장 좋게 평가한 영화를 찾는다. U1 이 아직 보지 않은 영화 M4, M5 에 대해 U3, U4 의 평점 평균을 내면 각각 2 와 5 이다. 따라서 평점 평균이 높은 M5를 U1도 좋아할 것으로 예상.

### MovieLens dataset
- MovieLens 라는 테스트 추천 시스템으로부터 수집된 데이터  
- 사용자들이 영화에 대해서 평가한 데이터  
- MovieLens에서는 사용자가 각 영화를 1점(최악)에서 5점(최고) 사이의 점수로 평가  

In [ ]:
# 필요한 라이브러리 임포트
# from sklearn.metrics import mean_squared_error  # sklearn 구 버전
# 두 벡터 간의 코사인 유사도를 계산하기 위한 함수
# 영화 데이터의 열 이름을 정의
# 영화 (특성) 정보 로드

- content-based filtering 과 달리 CF 는 similarity 를 이용하므로 **item의 feature 정보가 필요하지 않기 때문**에 movies dataframe에서 title 을 제외한 모든 다른 열을 drop 한다.  

In [ ]:
# item의 특성 정보가 필요하지 않으므로 movie ID와 title 빼고 다른 데이터 제거

In [ ]:
# 사용자의 영화 평점(rating) 정보 로드
# 평점 데이터의 열 이름을 정의
# 평점 데이터를 파일에서 읽어옴
# 로드된 평점 데이터의 첫 5개 행을 출력하여 데이터 확인

- ratings 만을 입력으로 사용

In [ ]:
# timestamp 제거 

In [ ]:
# ratings를 train, test set 으로 분리
# train set으로 평점 matrix 생성
# test set으로 모델 성능 평가   

### train 데이터로 평점 matrix 생성

In [ ]:
# 평점 매트릭스를 생성 - 사용자별로 각 영화에 대한 평점을 나타내는 2차원 테이블.
# 사용자 ID가 행 인덱스로, 영화 ID가 열 인덱스로 설정됩니다. 각 셀에는 해당 사용자가 해당 영화에 부여한 평점이 저장됩니다.

### User-Based CF 알고리즘

step 1) 모든 사용자간의 평가의 유사도를 계산  
step 2) 현재 추천 대상이 되는 사용자와 다른 사용자간의 유사도를 추출  
step 3) 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가값 계산. 예상 평가값은 다른 사용자의 해당 아이템에 대한 평가(평점)를 현재 사용자와 그 사용자의 유사도로 가중 평균  
step 4) 아이템 중에서 예상 평가값이 가장 높은 N 개의 아이템을 추천  

이 알고리즘이 작동하기 위해서는 추천 대상 사용자가 평가한 아이템과 다른 사용자들이 평가한 동일한 아이템이 유사하게 계산되면 신뢰할 수 있는 모델이 만들어진 것이고, 이 모델을 이용하여 추천 대상 사용자가 보지 않은 아이템(영화)의 평점을 계산하고 가장 예상 평점이 높은 N 개 아이템을 추천하면 된다.    

이때, 추천 대상 사용자와 가장 유사도가 높은 K 개의 다른 사용자의 평가만 반영하여 예측 평가의 품질을 높인다.

이를 위해 추천 사용자, 영화 id 를 parameter 로 받아 예상 평점을 반환하는 함수를 작성

### step 1) 모든 사용자간의 평가의 유사도를 계산

In [ ]:
# 평점 매트릭스의 복사본을 생성하고, NaN 값을 0으로 채웁니다. (평점을 부여하지 않은 영화는 평점이 0인 것으로 처리)
# 사용자 간의 코사인 유사도를 계산합니다.
# 여기서 matrix_dummy의 각 행은 사용자를 나타내며, 열은 영화에 대한 사용자의 평점을 나타냅니다.
# cosine_similarity 함수는 이 행렬을 입력으로 받아, 모든 가능한 사용자 쌍 간의 코사인 유사도를 계산합니다.
# 코사인 유사도 매트릭스를 DataFrame으로 변환하고, 인덱스와 컬럼 이름을 rating_matrix의 인덱스(사용자 ID)로 설정합니다.
# 따라서, 이 DataFrame의 각 셀은 특정 사용자 쌍 사이의 코사인 유사도를 나타냅니다.

### 추천 대상이 되는 사용자에게 보지 않은 영화를 추천할 때 사용자가 부여할 rating 예측

1. 추천 대상 영화에 대해 이미 평가를 내린 모든 사용자의 rating값 가져오기 --> 영화 ID를 기준으로 모든 사용자의 평점 값을 필터링  
3. 추천 대상 영화를 평가하지 않은 사용자의 index 가져오기
4. 대상 영화를 평가하지 않은 사용자의 rating (null) 제거 --> 추천 대상 영화를 평가한 사용자만 남김

### step2) 현재 추천 대상이 되는 사용자와 다른 사용자들의 유사도를 추출

In [ ]:
# 사용자별 영화 평점 테이블

- `ratings_matrix(사용자별로 각 영화에 대한 평점을 나타내는 테이블)`에서 user_id 1 이 movie_id 5를 평가하지 않았으므로 user_id 1 에게 movie_id 5을 추천할 때의 예상 rating을 예측해 봅니다.

In [ ]:
# 현재 추천 대상이 되는 사용자
# 추천대상 영화
#step2)현재 추천 대상이 되는 사용자와 다른 사용자들 간의 similarity 가져오기

In [ ]:
# step 3) 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가값 계산. 
# 1.추천대상 영화(movie_id)에 대한 모든 사용자의 평점을 구함
# rating_matrix에서 영화 ID가 movie_id인 열을 선택

In [ ]:
# 2.추천대상 영화를 평가하지 않은 사용자(movie_ratings에서 평점이 NaN)의 index 가져오기
# 추천대상 영화에 대해 평점을 부여하지 않은 사용자의 수
# 평점을 부여하지 않은 사용자의 인덱스 목록 출력

In [ ]:
# 3. 추천대상 영화를 평가하지 않은 사용자의 rating (null) 제거 --> 추천 대상 영화를 평가한 사용자만 남김
# 추천대상 영화에 평점을 부여한 사용자의 수

In [ ]:
# 4. 현재 영화를 평가하지 않은 사용자의 similarity값 제거
#추천대상 영화에 실제로 평점을 부여한 사용자들 간의 유사도 값의 수

In [ ]:
# 정규화: 유사도 점수의 합이 1이 되도록 조정

### 가장 유사도가 높은 상위 K명의 사용자 찾기

In [ ]:
# 5.유사도가 높은 사용자 순으로 정렬

In [ ]:
# 6.상위 n_neighbors에 해당하는 평점만 추출

### Implicit Rating을 이용한 추천 공식

$$\tilde{r}_{ui} = \frac{\sum_{v \in KNN(u)}r_{vi} \cdot S_{vu}}{\sum_{v \in KNN(u)}S_{vu}}$$

In [ ]:
# 7. 가중평균 계산
# top_users: 상위 유사도를 가진 사용자들의 유사도 점수 배열
# top_ratings: 상위 유사도를 가진 사용자들이 해당 영화에 매긴 평점 배열
# np.dot(top_users, top_ratings): 유사도 점수와 평점의 가중합을 계산
# top_users.sum(): 유사도 점수의 합으로 가중합을 나누어 가중평균을 계산
# 계산된 가중평균 평점 출력

###  특정 사용자에게 추천하는 영화(movie_id)의  가중평균 rating을 계산하는 함수
- 위의 과정을 model 함수로 작성
- 위에서 개별 사용자와 아이템에 대해 구하던 predicted_rating을 함수로 작성
- 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)

In [ ]:
def UBCF_knn_model(user_id, movie_id, rating_matrix, user_similarity, n_neighbors=0):
    # 현재 영화가 rating_matrix의 컬럼에 있는지 확인
        # 추천 대상 사용자와 다른 사용자 간의 유사도 가져오기
        # 1. 추천 대상이 되는 영화에 대한 모든 사용자 평가
        # 2. 추천 대상 영화를 평가하지 않은 사용자의 index 가져오기
        # 3. 추천 대상 영화를 평가하지 않은 사용자의 rating (null) 제거
        # 4. 추천 대상 영화를 평가하지 않은 사용자의 similarity값 제거
        # 유사도의 합으로 나누기 전에 유사도가 0인지 확인
        # k가 지정되지 않은 경우 모든 유사도를 사용
        # n_neighbors가 1 이상이면, 유사도가 높은 순으로 정렬하고 상위 n_neighbors만 사용
            # 5.유사도가 높은 사용자 순으로 정렬
            # 6.상위 n_neighbors에 해당하는 평점만 추출
            # 7.가중평균 계산
            # 유사한 사용자가 없거나 한 명뿐인 경우 기본 평점을 할당

### 모델 성능 평가 - 모델별 RMSE를 계산하는 함수 
- Test set을 이용하여 모델 성능 측정  

In [ ]:
def score(model, n_neighbors=0):
    # X_test 데이터셋에서 사용자 ID와 영화 ID의 쌍을 생성합니다.
    # 생성된 각 ID 쌍에 대해 모델을 사용하여 예상 평점을 계산합니다.
    # X_test 데이터셋에서 실제 평점을 가져옵니다.
    # 실제 평점과 예측 평점 사이의 RMSE를 계산하여 반환합니다.
    # return mean_squared_error(y_true, y_pred, squared=False)  # sklearn old version

In [ ]:
# 정확도 계산

### Step4) 특정 사용자에 대하여 영화 추천
- 이미 본 영화는 제외하고 보지 않은 영화 중에서 예상 평점이 높은 영화 추천

In [ ]:
# rating matrix에서 추천 대상 사용자의 해당 row 가져오기

In [ ]:
# 가져온 row에서 추천 대상 사용자의 평점이 없는(보지 않은) 영화를 식별합니다.

In [ ]:
# 평가되지 않은 각 영화에 대해 예상 평점을 계산합니다.
# 예상 평점순으로 정렬된 Series를 생성합니다.

영화를 높은 예상 평점에 따라 제목을 뽑아 줌

### 주어진 사용자에 대하여 추천 받는 함수
위 코드들을 하나의 함수로 작성 

In [ ]:
def recom_movie(user_id, n_items, n_neighbors=30):
    # rating matrix에서 추천 대상 사용자의 해당 row 가져오기
    # 가져온 row에서 추천 대상 사용자의 평점이 없는(보지 않은) 영화를 식별합니다.
    # 평가되지 않은 각 영화에 대해 예상 평점을 계산합니다.
    # 예상 평점순으로 정렬된 Series를 생성합니다.
    # 추천 영화의 제목을 가져옵니다.

In [ ]:
# 특정 사용자에 대해 n_items 개의 영화 추천

### 최적의 neighbor size 구하기

In [ ]:
# 다양한 이웃 크기(neighbor size)에 대해 RMSE 계산 및 출력